In [8]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageOps
from sklearn.neural_network import MLPRegressor
import pickle5 as pickle

from sklearn.model_selection import train_test_split

In [68]:
with open("bsc-3m/traj_01_elab.pkl", "rb") as f:
    df = pickle.load(f)

In [69]:
cols=["id","class","xs","ys", "xs_640x360", "ys_640x360", "times", "frame_in", "frame_out", "num_frames"]
data = df[cols]
data["class"].unique()

array(['Bicycle', 'Pedestrian', 'Car', 'Van', 'Light Truck', 'Bus',
       'Motorcycle', 'Heavy Vehicle'], dtype=object)

In [70]:
#id, class, xs, ys (or actually xs_640x360, ys_640x360 to plot it on the video/frame)
cols=["id","class","xs","ys", "xs_640x360", "ys_640x360", "times", "frame_in", "frame_out", "num_frames"]
data = df[cols]
data = data.loc[(data['class'] == 'Car') | (data['class'] == 'Van') | (data['class'] == 'Light Truck') | (data['class'] == 'Motorcycle')] # select cars

In [71]:
data

,id,class,xs,ys,xs_640x360,ys_640x360,times,frame_in,frame_out,num_frames
5,12,Car,"[6.1, 7.42, 8.76, 10.09, 11.43, 12.77, 14.11, ...","[445.6, 445.41, 445.23, 445.05, 444.87, 444.71...","[3.05, 3.71, 4.38, 5.045, 5.715, 6.385, 7.055,...","[222.8, 222.705, 222.615, 222.525, 222.435, 22...","[1.233337, 1.26667, 1.300004, 1.333337, 1.3666...",37,1235,1198
7,16,Car,"[1142.41, 1141.51, 1140.63, 1139.74, 1138.86, ...","[491.99, 491.93, 491.86, 491.78999999999996, 4...","[571.205, 570.755, 570.315, 569.87, 569.43, 56...","[245.995, 245.965, 245.93, 245.89499999999998,...","[7.333355, 7.366688, 7.400021, 7.433355, 7.466...",220,1346,1126
12,26,Van,"[27.71, 27.82, 27.9, 27.96, 28.02, 28.09, 28.1...","[442.82, 442.86, 442.88, 442.9, 442.92, 442.94...","[13.855, 13.91, 13.95, 13.98, 14.01, 14.045, 1...","[221.41, 221.43, 221.44, 221.45, 221.46, 221.4...","[32.400094, 32.433427, 32.466761, 32.500094, 3...",972,1283,311
18,38,Van,"[17.35, 22.82, 27.98, 32.98, 37.93, 42.99, 48....","[457.07, 456.35, 455.72, 455.19, 454.76, 454.4...","[8.675, 11.41, 13.99, 16.49, 18.965, 21.495, 2...","[228.535, 228.175, 227.86, 227.595, 227.38, 22...","[51.433482, 51.466816, 51.500149, 51.533483, 5...",1543,1703,160
20,42,Van,"[12.97, 17.93, 22.7, 27.35, 31.98, 36.69, 41.5...","[455.34, 454.55, 453.83, 453.19, 452.64, 452.1...","[6.485, 8.965, 11.35, 13.675, 15.99, 18.345, 2...","[227.67, 227.275, 226.915, 226.595, 226.32, 22...","[53.833489, 53.866823, 53.900156, 53.93349, 53...",1615,1801,186
...,...,...,...,...,...,...,...,...,...,...
18814,37630,Car,"[1182.71, 1181.25, 1179.79, 1178.34, 1176.88, ...","[478.15999999999997, 478.25, 478.3300000000000...","[591.355, 590.625, 589.895, 589.17, 588.44, 58...","[239.07999999999998, 239.125, 239.165000000000...","[36521.505847, 36521.53918, 36521.572514, 3652...",1095656,1095928,272
18815,37632,Car,"[8.18, 12.67, 16.85, 20.84, 24.79, 28.83, 33.0...","[455.04, 454.84, 454.67, 454.55, 454.48, 454.4...","[4.09, 6.335, 8.425, 10.42, 12.395, 14.415, 16...","[227.52, 227.42, 227.335, 227.275, 227.24, 227...","[36524.93919, 36524.972524, 36525.005857, 3652...",1095759,1095944,185
18816,37634,Car,"[1180.44, 1178.96, 1177.49, 1176.02, 1174.55, ...","[477.39, 477.3, 477.22, 477.15, 477.0900000000...","[590.22, 589.48, 588.745, 588.01, 587.275, 586...","[238.695, 238.65, 238.61, 238.575, 238.5450000...","[36525.539192, 36525.572525, 36525.605859, 365...",1095777,1097123,1346
18820,37642,Car,"[485.31, 485.31, 485.31, 485.31, 485.31, 485.3...","[584.58, 584.58, 584.58, 584.58, 584.58, 584.5...","[242.655, 242.655, 242.655, 242.655, 242.655, ...","[292.29, 292.29, 292.29, 292.29, 292.29, 292.2...","[36532.505879, 36532.539212, 36532.572546, 365...",1095986,1096253,267


In [72]:
for _ , n in df.iterrows():
    for i in range(len(n['xs'])):
        n['xs'][i]/=1280
        n['ys'][i]/=720

In [73]:
data

,id,class,xs,ys,xs_640x360,ys_640x360,times,frame_in,frame_out,num_frames
5,12,Car,"[0.004765625, 0.005796875, 0.00684375, 0.00788...","[0.6188888888888889, 0.618625, 0.618375, 0.618...","[3.05, 3.71, 4.38, 5.045, 5.715, 6.385, 7.055,...","[222.8, 222.705, 222.615, 222.525, 222.435, 22...","[1.233337, 1.26667, 1.300004, 1.333337, 1.3666...",37,1235,1198
7,16,Car,"[0.8925078125000001, 0.8918046875, 0.891117187...","[0.6833194444444445, 0.6832361111111112, 0.683...","[571.205, 570.755, 570.315, 569.87, 569.43, 56...","[245.995, 245.965, 245.93, 245.89499999999998,...","[7.333355, 7.366688, 7.400021, 7.433355, 7.466...",220,1346,1126
12,26,Van,"[0.0216484375, 0.021734375, 0.021796875, 0.021...","[0.6150277777777777, 0.6150833333333333, 0.615...","[13.855, 13.91, 13.95, 13.98, 14.01, 14.045, 1...","[221.41, 221.43, 221.44, 221.45, 221.46, 221.4...","[32.400094, 32.433427, 32.466761, 32.500094, 3...",972,1283,311
18,38,Van,"[0.0135546875, 0.017828125, 0.021859375, 0.025...","[0.6348194444444444, 0.6338194444444445, 0.632...","[8.675, 11.41, 13.99, 16.49, 18.965, 21.495, 2...","[228.535, 228.175, 227.86, 227.595, 227.38, 22...","[51.433482, 51.466816, 51.500149, 51.533483, 5...",1543,1703,160
20,42,Van,"[0.010132812500000001, 0.0140078125, 0.0177343...","[0.6324166666666666, 0.6313194444444444, 0.630...","[6.485, 8.965, 11.35, 13.675, 15.99, 18.345, 2...","[227.67, 227.275, 226.915, 226.595, 226.32, 22...","[53.833489, 53.866823, 53.900156, 53.93349, 53...",1615,1801,186
...,...,...,...,...,...,...,...,...,...,...
18814,37630,Car,"[0.9239921875, 0.9228515625, 0.9217109375, 0.9...","[0.6641111111111111, 0.6642361111111111, 0.664...","[591.355, 590.625, 589.895, 589.17, 588.44, 58...","[239.07999999999998, 239.125, 239.165000000000...","[36521.505847, 36521.53918, 36521.572514, 3652...",1095656,1095928,272
18815,37632,Car,"[0.006390625, 0.0098984375, 0.0131640625, 0.01...","[0.632, 0.6317222222222222, 0.6314861111111111...","[4.09, 6.335, 8.425, 10.42, 12.395, 14.415, 16...","[227.52, 227.42, 227.335, 227.275, 227.24, 227...","[36524.93919, 36524.972524, 36525.005857, 3652...",1095759,1095944,185
18816,37634,Car,"[0.9222187500000001, 0.9210625, 0.9199140625, ...","[0.6630416666666666, 0.6629166666666667, 0.662...","[590.22, 589.48, 588.745, 588.01, 587.275, 586...","[238.695, 238.65, 238.61, 238.575, 238.5450000...","[36525.539192, 36525.572525, 36525.605859, 365...",1095777,1097123,1346
18820,37642,Car,"[0.3791484375, 0.3791484375, 0.3791484375, 0.3...","[0.8119166666666667, 0.8119166666666667, 0.811...","[242.655, 242.655, 242.655, 242.655, 242.655, ...","[292.29, 292.29, 292.29, 292.29, 292.29, 292.2...","[36532.505879, 36532.539212, 36532.572546, 365...",1095986,1096253,267


### features
    x_coordinate x # normalize
    y_coordinate x
    x_direction  x more!
    y_direction  x
    x_dest       x
    y_dest       x
    light_color # how
    surrounding_traffic?
    time?

### target
    next_x       x as vector please
    next_y       x

In [74]:
def format_data(data, step_size):
    # position
    x_pos = np.array([])
    y_pos = np.array([])

    # target position
    x_tar = np.array([])
    y_tar = np.array([])

    # direction vectors
    x_vec = np.array([])
    y_vec = np.array([])
    
    # direction vectors
    x_vec2 = np.array([])
    y_vec2 = np.array([])

    # destination
    x_dest = np.array([])
    y_dest = np.array([])

    # times
    times = np.array([])

    # euclidean distance
    euc = np.array([])
    
    # frames
    frames = np.array([])
    
    # id
    ids = np.array([])
    
    for i in range(len(data)):
        # current row
        a = np.array(data['xs'].iloc[i][::step_size])
        b = np.array(data['ys'].iloc[i][::step_size])
        if len(a) <= 2:
            continue
        

        # compute position
        x_pos = np.concatenate((x_pos, a[:-1]))
        y_pos = np.concatenate((y_pos, b[:-1]))

        # compute target vector
        x_tar = np.concatenate((x_tar, a[1:]-a[:-1]))
        y_tar = np.concatenate((y_tar, b[1:]-b[:-1]))

        # compute direction vectors
        x_prev = np.concatenate(([a[0]], a[:-2]))
        y_prev = np.concatenate(([b[0]], b[:-2]))
        x_vec = np.concatenate((x_vec, (a[:-1] - x_prev)))
        y_vec = np.concatenate((y_vec, (b[:-1] - y_prev)))
        
        # compute direction vectors
        x_prev2 = np.concatenate(([x_prev[0]], x_prev[:-1]))
        y_prev2 = np.concatenate(([y_prev[0]], y_prev[:-1]))
        x_vec2 = np.concatenate((x_vec2, (x_prev - x_prev2)))
        y_vec2 = np.concatenate((y_vec2, (y_prev - y_prev2)))

        # destination
        x_dest = np.concatenate((x_dest, [a[-1]]*(len(a)-1)))
        y_dest = np.concatenate((y_dest, [b[-1]]*(len(b)-1)))
        
        # euclidean
        euc_x = np.array([a[-1]]*len(a)) - a
        euc_y = np.array([b[-1]]*len(b)) - b
        euc_i = np.linalg.norm(np.vstack((euc_x, euc_y)), axis=0)[:-1]
        euc = np.concatenate((euc, euc_i))
        
        # ids
        ids = np.concatenate((ids, np.array([int(data['id'].iloc[i])] * (len(a)-1))))
        
    df = pd.DataFrame({
                        'id' : ids, 
                        'x_pos' : x_pos, 'y_pos' : y_pos,
                        'x_vec' : x_vec, 'y_vec' : y_vec,
                        'x_vec2' : x_vec2, 'y_vec2' : y_vec2,
                        'x_tar' : x_tar, 'y_tar' : y_tar,
                        'x_dest' : x_dest, 'y_dest' : y_dest,
                        'euc' : euc
                      })
    
    df = df.loc[df['euc']!=0]
    
    return df

In [75]:
df_train, df_val = train_test_split(data, test_size=0.2, random_state=1)

df_out = format_data(df_train, 15)
with open('train.pkl', 'wb') as f:
    pickle.dump(df_out, f)


df_out = format_data(df_val, 15)
with open('val.pkl', 'wb') as f:
    pickle.dump(df_out, f)